In [1]:
#To do 07/02/2020: 914*  714 catnum still not working. add rend ="del" AND -1404. 3 add rend didn't work.

#To extract page numbers(pb), div2 label, and catnum from Misc XML file and save as csv.
#To ignore digits from <add rend="del"> and <add rend="pencil"> which occurs once with catnum 2082
#from <catnum>, remove: -; ____; ?; .; and a tick in front of a number. 
#Excludes non-Sloane cat entries.
from lxml import etree
import re
import csv

In [2]:
path = '/Users/deborahleem/Documents/PhD/SloaneDBwork19/Miscellanies5Ah_MASTER_fixed_whitespaceDL.xml'
doc = etree.parse(open(path))
root = doc.getroot()

In [3]:
def remove_element(el):
    parent = el.getparent()
    if el.tail:
        prev = el.getprevious()
        if prev:
            prev.tail = (prev.tail or '') + el.tail
        else:
            parent.text = (parent.text or '') + el.tail
    parent.remove(el)

# Remove every <add rend="del"> and <add rend="pencil"> from document before the rest of analysis
for e in list(root.iterfind('.//{*}add')):
    if ('rend' in e.attrib) and (e.attrib['rend'] == 'del' or e.attrib['rend'] == 'pencil'):
        #print('removing', etree.tostring(e, pretty_print=True))
        if e.text == '-1861' or e.text == '2094.' or e.text == '-382.':
            continue
        remove_element(e)


/Users/deborahleem/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  """


In [4]:
#Catnums that need to be excluded as they are not part of Sloane
#<pb xml:id="EPF108491027v"/>, Miscellanies, 2108, 2109, 2110, 2111
#<pb xml:id="EPF108491177v"/>, Antiquities, 1,2
#<pb xml:id="EPF108491178v"/>, Antiquities, 1,2,3,4
#<pb xml:id="EPF108491179v"/>, Antiquities, 4,5,6,7,8,1
#<pb xml:id="EPF108491180v"/>, Antiquities, 2,3
#<pb xml:id="EPF108491181v"/>, Antiquities, 4,5
#<pb xml:id="EPF108491182v"/>, Antiquities, 1,2,3,4,5
#<pb xml:id="EPF108491183v"/>, Antiquities, 6,7,8,9,10,11,12
#<pb xml:id="EPF108491184v"/>, Antiquities, 13,14,1,2,3,4,5,1
#<pb xml:id="EPF108491186v" type="Bronzes"/>, Antiquities, 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
#<pb xml:id="EPF108491187v" type="Bronzes"/>, Antiquities, 20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
#<pb xml:id="EPF108491188v"/>, Antiquities, 40,41,42,43,44,45,46,47,48,1,2,3,4,5,6,7,8,9
#<pb xml:id="EPF108491189v"/>, Antiquities, 10,11,12,1,2,3,4,5,6
#<pb xml:id="EPF108491190v"/>, Antiquities, 7,8,9,10,11

In [5]:
excluded = {
  "EPF108491027v": ["2108","2109","2110","2111"], # "Miscellanies"
  "EPF108491177v": ["1","2"], # "Antiquities"
  "EPF108491178v": ["1","2","3","4"], # "Antiquities"
  "EPF108491179v": ["4","5","6","7","8","1"], # "Antiquities"
  "EPF108491180v": ["2","3"], # "Antiquities"
  "EPF108491181v": ["4","5"], # "Antiquities"
  "EPF108491182v": ["1","2","3","4","5"], # "Antiquities"
  "EPF108491183v": ["6","7","8","9","10","11","12"], # "Antiquities"
  "EPF108491184v": ["13","14","1","2","3","4","5","1"], # "Antiquities"
  "EPF108491186v": ["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19"], # "Antiquities"
  "EPF108491187v": ["20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39"], # "Antiquities"
  "EPF108491188v": ["40","41","42","43","44","45","46","47","48","1","2","3","4","5","6","7","8","9"], # "Antiquities"
  "EPF108491189v": ["10","11","12","1","2","3","4","5","6"], # "Antiquities"
  "EPF108491190v": ["7","8","9","10","11"], # "Antiquities"
}


In [6]:
def find_div2_label(element):
    while element is not None:
        if element.tag == '{http://www.tei-c.org/ns/1.0}div2':
            label = element.find('{*}label')
            if label is not None:
                return label.text
            return
        element = element.getparent()

In [7]:
def find_enclosing_div3(element):
    while element is not None:
        if element.tag == '{http://www.tei-c.org/ns/1.0}div3':
            return element
        element = element.getparent()
    
def find_pb_xml_id(element):
    element = find_enclosing_div3(element)
    while element is not None:
#         print(element.tag)
        if element.tag == '{http://www.tei-c.org/ns/1.0}pb':
            return element.attrib['{http://www.w3.org/XML/1998/namespace}id']
        element = element.getprevious()
                
for catnum in root.iterfind('.//{*}catnum'):
    print(find_pb_xml_id(catnum))


EPF108491002v
EPF108491002v
EPF108491002v
EPF108491002v
EPF108491002v
EPF108491002v
EPF108491002v
EPF108491002v
EPF108491002v
EPF108491002v
EPF108491002v
EPF108491002v
EPF108491002v
EPF108491002v
EPF108491002v
EPF108491002v
EPF108491002v
EPF108491002v
EPF108491002v
EPF108491002v
EPF108491002v
EPF108491003
EPF108491003
EPF108491003
EPF108491003
EPF108491003
EPF108491003
EPF108491003
EPF108491003
EPF108491003
EPF108491003
EPF108491003
EPF108491003
EPF108491003v
EPF108491003v
EPF108491003v
EPF108491003v
EPF108491003v
EPF108491003v
EPF108491003v
EPF108491003v
EPF108491003v
EPF108491003v
EPF108491003v
EPF108491003v
EPF108491003v
EPF108491003v
EPF108491003v
EPF108491003v
EPF108491003v
EPF108491003v
EPF108491003v
EPF108491003v
EPF108491003v
EPF108491003v
EPF108491003v
EPF108491003v
EPF108491003v
EPF108491003v
EPF108491003v
EPF108491003v
EPF108491004v
EPF108491004v
EPF108491004v
EPF108491004v
EPF108491004v
EPF108491004v
EPF108491004v
EPF108491004v
EPF108491004v
EPF108491004v
EPF108491004v
EPF1

EPF108491140
EPF108491140
EPF108491140
EPF108491140
EPF108491140
EPF108491140
EPF108491140
EPF108491140
EPF108491140
EPF108491140
EPF108491140
EPF108491140
EPF108491140
EPF108491140
EPF108491141
EPF108491141
EPF108491141
EPF108491141
EPF108491141
EPF108491141
EPF108491141
EPF108491141
EPF108491141
EPF108491141
EPF108491141
EPF108491141
EPF108491141
EPF108491141
EPF108491141
EPF108491141
EPF108491141
EPF108491141
EPF108491141
EPF108491141
EPF108491142
EPF108491142
EPF108491142
EPF108491142
EPF108491142
EPF108491142
EPF108491142
EPF108491142
EPF108491142
EPF108491142
EPF108491142
EPF108491142
EPF108491142
EPF108491142
EPF108491142
EPF108491142
EPF108491142av
EPF108491143
EPF108491143
EPF108491143
EPF108491143
EPF108491143
EPF108491143
EPF108491143
EPF108491143
EPF108491143
EPF108491143
EPF108491143
EPF108491143
EPF108491143
EPF108491143
EPF108491143
EPF108491143
EPF108491143
EPF108491143
EPF108491144
EPF108491144
EPF108491144
EPF108491144
EPF108491144
EPF108491144
EPF108491144
EPF1084911

EPF108491207
EPF108491207
EPF108491207
EPF108491208
EPF108491208
EPF108491208
EPF108491208
EPF108491208
EPF108491208
EPF108491208
EPF108491208
EPF108491208
EPF108491208
EPF108491208
EPF108491208
EPF108491208
EPF108491209
EPF108491209
EPF108491209
EPF108491209
EPF108491209
EPF108491209
EPF108491209
EPF108491209
EPF108491210
EPF108491210
EPF108491210
EPF108491210
EPF108491210
EPF108491210
EPF108491210
EPF108491210
EPF108491210
EPF108491210
EPF108491210
EPF108491210
EPF108491210
EPF108491210
EPF108491210
EPF108491210
EPF108491211
EPF108491211
EPF108491211
EPF108491211
EPF108491211
EPF108491211
EPF108491211
EPF108491211
EPF108491211
EPF108491211
EPF108491211
EPF108491211
EPF108491211
EPF108491212
EPF108491212
EPF108491212
EPF108491212
EPF108491212
EPF108491212
EPF108491212
EPF108491212
EPF108491212
EPF108491212
EPF108491213
EPF108491213
EPF108491213
EPF108491213
EPF108491213
EPF108491213
EPF108491213
EPF108491213
EPF108491213
EPF108491213
EPF108491214
EPF108491214
EPF108491214
EPF108491214

In [8]:
def cleanup_catnum(text):
    #removing from beginning and end:
    # \s spaces/tabs/newlines
    # \- dashes
    # . dots
    # ? question marks
    # _ underscores
    # ✓
    text = re.sub(r'^[ ✓\t\n.\-_?]+', "", text)
    text = re.sub(r'[ ✓\t\n.\-_?]+$', "", text)
    text = re.sub(r'[ ✓\t\n.\-_?]+\*', "*", text)
    text = text.replace('. ', ' ')
    return text

results = []
for catnum in root.iterfind('.//{*}catnum'):
    text = "".join(catnum.itertext())
    text = cleanup_catnum(text)
    label = find_div2_label(catnum)
    pb_xml_id = find_pb_xml_id(catnum)
    if pb_xml_id in excluded:
        if text in excluded[pb_xml_id]:
#             print('exclude', pb_xml_id, text)
            continue
#         else:
#             print('do not exclude', pb_xml_id, text)
    
    results.append([text, label, pb_xml_id])
    #print(etree.tostring(catnum))
    #print([text, label])

In [9]:
# results0 = results

In [10]:
#Showing all changes.
# [(results0[i], results[i]) for i in range(len(results)) if results0[i] != results[i]]

In [11]:
# All entries that are not just digits and start
[[x, results.index(x)] for x in results if not x[0].replace('*', '').isdigit()]

[[['No 1', 'Miscellanies.', 'EPF108491009v'], 238],
 [['1933 No 7', 'Miscellanies.', 'EPF108491010v'], 259],
 [['1933 No 8', 'Miscellanies.', 'EPF108491010v'], 260],
 [['1933 No 9', 'Miscellanies.', 'EPF108491010v'], 261],
 [['1933 No 10', 'Miscellanies.', 'EPF108491010v'], 262],
 [['1933 No 11', 'Miscellanies.', 'EPF108491010v'], 263],
 [['1933 No 12', 'Miscellanies.', 'EPF108491010v'], 264],
 [['1933 No 13', 'Miscellanies.', 'EPF108491010v'], 265],
 [['1933 No 14', 'Miscellanies.', 'EPF108491010v'], 266],
 [['1933 No 15', 'Miscellanies.', 'EPF108491010v'], 267],
 [['1933 No 16', 'Miscellanies.', 'EPF108491010v'], 268],
 [['1933 No 17', 'Miscellanies.', 'EPF108491010v'], 269],
 [['1933 No 18', 'Miscellanies.', 'EPF108491010v'], 270],
 [['1933 No 20', 'Miscellanies.', 'EPF108491010v'], 271],
 [['1933 No 21', 'Miscellanies.', 'EPF108491010v'], 272],
 [['1933 No 22', 'Miscellanies.', 'EPF108491010v'], 273],
 [['1933 No 25', 'Miscellanies.', 'EPF108491010v'], 274],
 [['1933 No 26', 'Misce

In [13]:
with open('all_catnums_and_their_div2_labels.csv', 'w') as file:
    writer = csv.writer(file)
    for catnum, label, pageid in results:
        writer.writerow([catnum, label, pageid])